In [3]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import data_loader
import data_references
from energy_channels import EnergyChannel
import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import plot_tools
import numpy as np
import pandas as pd
import psd_tool
from field_models import model
import data_cache
import scipy


import importlib
importlib.reload(plot_tools)
importlib.reload(data_loader)
importlib.reload(data_references)
importlib.reload(psd_tool)
importlib.reload(data_cache)

%matplotlib qt

In [4]:
start = datetime.datetime(year = 2017, month = 7, day = 1)
end = datetime.datetime(year = 2017, month = 10, day = 1)

l_cut = 3.75
tol = 0.15

In [59]:
data_A = data_loader.load_compressed_rept_data(satellite="A", start=start, end=end)
data_B = data_loader.load_compressed_rept_data(satellite="B", start=start, end=end)

fesa_A = data_A["FESA"]
L_A = data_A["L"]
mlt_A = data_A["MLT"]
epoch_A = data_A["EPOCH"]
energies = data_A["ENERGIES"]

fesa_B = data_B["FESA"]
L_B = data_B["L"]
mlt_B = data_B["MLT"]
epoch_B = data_B["EPOCH"]
energies = data_B["ENERGIES"]

Loading REPT data between: 2017-04-01 00:00:00 and 2017-07-01 00:00:00.
Loading : REPT_201704_A.npz
Loading : REPT_201705_A.npz
Loading : REPT_201706_A.npz
Loading : REPT_201707_A.npz
Loading REPT data between: 2017-04-01 00:00:00 and 2017-07-01 00:00:00.
Loading : REPT_201704_B.npz
Loading : REPT_201705_B.npz
Loading : REPT_201706_B.npz
Loading : REPT_201707_B.npz


In [4]:
satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))

fesa_A = fesa_A[:, (energies < 4.3)]
fesa_B = fesa_B[:, (energies < 4.3)]

fesa_A_DF = pd.DataFrame(fesa_A[satisfies_L_cut_A, :], index=epoch_A[satisfies_L_cut_A])
fesa_B_DF = pd.DataFrame(fesa_B[satisfies_L_cut_B, :], index=epoch_B[satisfies_L_cut_B])

In [5]:
fesa = pd.concat([fesa_A_DF, fesa_B_DF], sort=False).sort_index()
fesa = fesa.resample("5h").mean()
#FESA = FESA.rolling(window=avg_t, center=True).mean()

In [5]:
omni_1_min = data_loader.load_omni_data_1min_res(start=start, end=end)
omni_1_min = pd.DataFrame(omni_1_min)
omni_1_min = omni_1_min.set_index(keys="EPOCH", drop=True)

omni_1_hour = data_loader.load_omni_data_1hour_res(start=start, end=end)
omni_1_hour = pd.DataFrame(omni_1_hour)
omni_1_hour = omni_1_hour.set_index(keys="EPOCH", drop=True)

Loading OMNI data between: 2017-07-01 00:00:00 and 2017-10-01 00:00:00
Loaded OMNI Data for : 2017-07-01 00:00:00
Loaded OMNI Data for : 2017-08-01 00:00:00
Loaded OMNI Data for : 2017-09-01 00:00:00
Loaded OMNI Data for : 2017-10-01 00:00:00
Loading: c:\Dev\Research\REPT_Enhancements_Tool\raw_data\OMNI\_1_hour_res\omni2_h0_mrg1hr_20170101_v01.cdf
Loading: c:\Dev\Research\REPT_Enhancements_Tool\raw_data\OMNI\_1_hour_res\omni2_h0_mrg1hr_20170701_v01.cdf


In [6]:
metop1 = data_loader.load_compressed_poes_data("metop1", start=start, end=end)
metop2 = data_loader.load_compressed_poes_data("metop2", start=start, end=end)
noaa15 = data_loader.load_compressed_poes_data("noaa15", start=start, end=end)
noaa16 = data_loader.load_compressed_poes_data("noaa16", start=start, end=end)
noaa18 = data_loader.load_compressed_poes_data("noaa18", start=start, end=end)
noaa19 = data_loader.load_compressed_poes_data("noaa19", start=start, end=end)

poes_satellites = [metop1, metop2, noaa15, noaa16, noaa18, noaa19]


Loading : POES_201707_metop1_CLEAN.npz
Loading : POES_201708_metop1_CLEAN.npz
Loading : POES_201709_metop1_CLEAN.npz
Loading : POES_201710_metop1_CLEAN.npz
Loading : POES_201707_metop2_CLEAN.npz
Loading : POES_201708_metop2_CLEAN.npz
Loading : POES_201709_metop2_CLEAN.npz
Loading : POES_201710_metop2_CLEAN.npz
Loading : POES_201707_noaa15_CLEAN.npz
Loading : POES_201708_noaa15_CLEAN.npz
Loading : POES_201709_noaa15_CLEAN.npz
Loading : POES_201710_noaa15_CLEAN.npz

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\noaa16\POES_201707_noaa16_CLEAN.npz, continuing...!

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\noaa16\POES_201708_noaa16_CLEAN.npz, continuing...!

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\noaa16\POES_201709_noaa16_CLEAN.npz, continuing...!

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\noaa16\POES_201710_noaa1

In [7]:
field_model = model.TS04D

B_l = 0.311653 / (l_cut ** 3)
E_l = [1.8, 2.1, 2.6, 3.4, 4.2]
M_e = scipy.constants.physical_constants["electron mass energy equivalent in MeV"][0]

mu_vals = [(E**2 + 2 * M_e * E) / (2 * M_e * B_l) for E in E_l]
chosen_k = 0.05

In [80]:
psd_refs_A = data_loader.load_psd(satellite = "A", field_model = field_model, start = start, end = end)
psd_refs_B = data_loader.load_psd(satellite = "B", field_model = field_model, start = start, end = end)

2017-10-01 00:00:00
Loading : PSD_201710_A_TS04D.npz
2017-11-01 00:00:00
Loading : PSD_201711_A_TS04D.npz
2017-12-01 00:00:00
Loading : PSD_201712_A_TS04D.npz
2017-10-01 00:00:00
Loading : PSD_201710_B_TS04D.npz
2017-11-01 00:00:00
Loading : PSD_201711_B_TS04D.npz
2017-12-01 00:00:00
Loading : PSD_201712_B_TS04D.npz


In [81]:
for chosen_mu in mu_vals:
    
    print(f"Selecting PSD for A at mu = {chosen_mu}, k = {chosen_k}")
    
    selected_psd_refs_A = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_A, chosen_mu=chosen_mu, chosen_k=chosen_k)
    
    print(f"Selecting PSD for B at mu = {chosen_mu}, k = {chosen_k}")

    selected_psd_refs_B = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_B, chosen_mu=chosen_mu, chosen_k=chosen_k)

    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_A, satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_B, satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)


Selecting PSD for A at mu = 841.0110060362525, k = 0.05
Time taken for loop: 15.568342447280884
Selecting PSD for B at mu = 841.0110060362525, k = 0.05
Time taken for loop: 15.725036144256592
Selecting PSD for A at mu = 1085.486406814505, k = 0.05
Time taken for loop: 15.33869218826294
Selecting PSD for B at mu = 1085.486406814505, k = 0.05
Time taken for loop: 15.71222186088562
Selecting PSD for A at mu = 1559.1720111415677, k = 0.05
Time taken for loop: 15.304744243621826
Selecting PSD for B at mu = 1559.1720111415677, k = 0.05
Time taken for loop: 15.693535327911377
Selecting PSD for A at mu = 2489.258145942802, k = 0.05
Time taken for loop: 15.521064519882202
Selecting PSD for B at mu = 2489.258145942802, k = 0.05
Time taken for loop: 15.92196249961853
Selecting PSD for A at mu = 3631.2694104399566, k = 0.05
Time taken for loop: 15.453933715820312
Selecting PSD for B at mu = 3631.2694104399566, k = 0.05
Time taken for loop: 15.585880756378174


In [22]:

slow_moving = omni_1_min["AE"].rolling("3h", center=False).mean()
#f = (OMNI["AE"] - slow_moving)
f = omni_1_min["AE"].diff()
threshold = 50
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings = zero_crossings.rolling("3h", center=False).sum().rolling("3h", center=True).mean()

fig, axs = plt.subplots(6, 1, sharex=True)

num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5)
    
axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))

    #satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
    #satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, linewidth=2, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)")
    
axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].set_title(rf"RBSP-COMBINED, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}")
axs[1].set_ylabel("PSD (c/Mev/G)")
axs[1].legend(loc = "upper right")

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[2].set_ylabel("# Crossing Threshold")
axs[2].set_ylim(bottom=0)
#axs[1].hlines(y=100, xmin=start, xmax=end, color="green", label="Enhancement Trigger")

axs[3].plot(POES.index, POES["Chorus"], color="black")
axs[3].set_ylabel("Naive Chorus (pT)")


axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].plot(omni_1_min.index, omni_1_min["AE"], color="black", label="AE")
axs[4].plot(omni_1_min.index, slow_moving, color="blue", linewidth=2, label="Slow Average")
axs[4].set_ylabel("AE (nT)")
axs[4].set_ylim(bottom=0)
axs[4].legend(loc = "upper right")

axs[5].yaxis.grid(color='gray', linestyle='dashed')
axs[5].xaxis.grid(color='gray', linestyle='dashed')
axs[5].plot(omni_1_hour.index, omni_1_hour["DST"], color="black")
axs[5].set_ylabel("DST")
axs[5].set_xlabel("Time")


#axs[4].plot(omni.index, f, color="black", label="$|{\delta}AE|$")
#axs[4].set_ylabel("$|{\delta}AE|$ (nT)")
#axs[4].yaxis.grid(color='gray', linestyle='dashed')
#axs[4].xaxis.grid(color='gray', linestyle='dashed')
#axs[4].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
#axs[4].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")

#axs[4].set_ylim(bottom=0)
#axs[4].legend()

#cbar = plt.colorbar(image_cmap, ax=axs[4])

#cbar.set_label("Bw (pT)\n", loc="center", rotation=270)

fig.set_size_inches(12, 8)

plt.show()

C:\Users\brofe\AppData\Local\Temp\ipykernel_32316\674101251.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)


In [31]:

f = omni_1_min["Bz"]
threshold = 1.0

zero_crossings = (((f.shift() >= 0) & (f < 0)) | ((f.shift() < 0) & (f >= 0))) & (f.abs() > threshold) & (f.shift().abs() > threshold)
num_crossings = zero_crossings.rolling("24h", center=True).sum().resample("5h").mean()

fig, axs = plt.subplots(6, 1, sharex="all")

num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5)


axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)", linewidth=2.5)
    
    
axs[1].set_title(rf"RBSP-COMBINED, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}")
axs[1].set_ylabel("PSD (c/Mev/G)")
axs[1].legend(loc="upper right")

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[2].set_ylabel("# Crossing Threshold")
#axs[2].set_ylim(bottom=0)

axs[3].plot(POES.index, POES["Chorus"], color="black")
axs[3].set_ylabel("Naive Chorus (pT)")
axs[3].set_xlabel("Time")

'''axs[4].plot(omni.index, f, color="black")

axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].set_ylabel("${\delta}B_{z}$ (nT)")
axs[4].hlines(y=0, xmin=start, xmax=end, color="black")
axs[4].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
axs[4].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")'''

axs[4].plot(omni_1_min.index, omni_1_min["Bz"], color="black")
#axs[4].plot(omni_1_min.index, omni_1_min["Bz"].rolling("15min", center=True).mean(), label="Slow Average", color="red")

axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].set_ylabel("$B_{z}$ (nT)")
axs[4].set_ylim(-15, 15)
axs[4].legend(loc = "upper right")

axs[5].yaxis.grid(color='gray', linestyle='dashed')
axs[5].xaxis.grid(color='gray', linestyle='dashed')
axs[5].plot(omni_1_hour.index, omni_1_hour["DST"], color="black")
axs[5].set_ylabel("DST")
axs[5].set_xlabel("Time")

plt.show()



<>:81: SyntaxWarning: invalid escape sequence '\d'
C:\Users\brofe\AppData\Local\Temp\ipykernel_32316\1611077060.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [8]:
f = omni_1_min["Bz"]
threshold = 1.0

zero_crossings = (((f.shift() >= 0) & (f < 0)) | ((f.shift() < 0) & (f >= 0))) & ((f - f.shift()).abs() > threshold)
num_crossings = zero_crossings.rolling("24h", center=True).sum().resample("5h").mean()

fig, axs = plt.subplots(5, 1, sharex="all")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[0].semilogy(PSD.index, PSD, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)", linewidth=2.5)


axs[0].set_title(rf"L* = {l_cut}, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}, RBSP-COMBINED")
axs[0].set_ylabel("PSD $(c/MeV/cm)^{3}$")
axs[0].legend(loc="upper right")
axs[0].set_xlim(start, end)


axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].plot(omni_1_hour.index, omni_1_hour["DST"], color="black")
axs[1].set_ylabel("DST (nT)")

axs[2].plot(omni_1_min.index, omni_1_min["Bz"], color="black")
#axs[4].plot(omni_1_min.index, omni_1_min["Bz"].rolling("15min", center=True).mean(), label="Slow Average", color="red")

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].set_ylabel("$B_{z}$ (nT)")
#axs[2].set_ylim(-15, 15)
axs[2].legend(loc = "upper right")

#axs[3].yaxis.grid(color='gray', linestyle='dashed')
#axs[3].xaxis.grid(color='gray', linestyle='dashed')
#axs[3].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
#axs[3].set_ylabel("# Crossing Threshold")
#axs[2].set_ylim(bottom=0)


axs[3].yaxis.grid(color='gray', linestyle='dashed')
axs[3].xaxis.grid(color='gray', linestyle='dashed')
axs[3].plot(omni_1_min.index, omni_1_min["AE"], color="black", label="AE")
axs[3].set_ylabel("AE (nT)")
axs[3].set_ylim(bottom=0)

#Parameters
L_bins = np.arange(3.0, 7.5, 0.25)
epoch_bin_size = "6h"


epoch_start = pd.Timestamp(start)
epoch_end = pd.Timestamp(end + pd.Timedelta(epoch_bin_size))
epoch_bins = pd.to_datetime(np.arange(epoch_start, epoch_end, pd.Timedelta(epoch_bin_size)))


total = pd.concat([pd.DataFrame({"Timestamp" : satellite.get_data(data_references.POES.EPOCH),
                                "L": satellite.get_data(data_references.POES.L), 
                                "MLT" : satellite.get_data(data_references.POES.MLT), 
                                "CHORUS" : satellite.get_data(data_references.POES.NAIVE_CHORUS_AMPLITUDES),
                                "Satid" : satellite.satid}) for satellite in poes_satellites], axis=0, ignore_index=True, verify_integrity=False, sort=False, copy=False)

total = total.drop(total[total["L"] <= min(L_bins)].index)
total = total.drop(total[total["L"] > max(L_bins)].index)
 
 
total["L_BIN"] = pd.cut(total["L"], L_bins, labels=False)
total["DATE_BIN"] = pd.cut(total["Timestamp"], epoch_bins, labels=False)
    
group_df = total.groupby(["L_BIN", "DATE_BIN"], dropna=True)

averages = np.zeros((len(L_bins) - 1, len(epoch_bins) - 1))

for i in range(averages.shape[0]):
    for j in range(averages.shape[1]):
        
        if (i, j) in group_df[["CHORUS"]].groups:
        
            averages[i, j] = group_df[["CHORUS"]].get_group((i, j)).mean()["CHORUS"]
            
        else:
            
            averages[i, j] = np.NaN

averages = averages + 1e-10 #Stabilize the log norm

image_cmap = axs[4].imshow(averages, 
                       origin="lower", 
                       extent=[epoch_bins[0], epoch_bins[-1], L_bins[0], L_bins[-1]],
                       norm=colors.LogNorm(vmin=1, vmax=100),
                       aspect="auto",
                       interpolation="none")

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

axins = inset_axes(
    axs[4],
    width="1%",  # width: 5% of parent_bbox width
    height="100%",  # height: 50%
    loc="lower left",
    bbox_to_anchor=(1.01, 0., 1, 1),
    bbox_transform=axs[4].transAxes,
    borderpad=0,
)

cbar = fig.colorbar(image_cmap, cax=axins)

cbar.set_label("Chorus Amplitude (pT)\n", loc="center", labelpad=15, rotation=270)


axs[4].set_ylabel(f"L")
axs[4].set_aspect('auto')
axs[4].set_xlabel("Time")




plt.show()


C:\Users\brofe\AppData\Local\Temp\ipykernel_5900\2394054073.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  mu_colors = plt.cm.get_cmap("plasma", num_mu)
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
C:\Users\brofe\AppData\Local\Temp\ipykernel_5900\2394054073.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total = pd.concat([pd.DataFrame({"Timestamp" : satellite.get_data(data_references.POES.EPOCH),
